In [5]:
import torch
import torchvision
import construct
import prep
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

ModuleNotFoundError: No module named 'fmpm'

In [2]:
image_dir = 'data/raman_images'
labels_file = 'data/labels.csv'

DATA = prep.prep_data(pd.read_csv(labels_file), image_dir)

transforms = torchvision.transforms.Compose([
                            torchvision.transforms.ToPILImage(),
                            torchvision.transforms.RandomRotation((-180,180)),
                            torchvision.transforms.CenterCrop((325)),
                            torchvision.transforms.ToTensor()
                                      ])

/Users/Will/GradSchool/CHEME/Project/FmPM/fmpm/will/prep.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  identification[i] = False
/Users/Will/GradSchool/CHEME/Project/FmPM/fmpm/will/prep.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  identification[i] = True


In [ ]:
data = prep.tenX_dataset(DATA, 'data/raman_images', transform=transforms)
criterion = torch.nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test, train = torch.utils.data.random_split(data, [int(.15*len(data)), len(data)-int(.15*len(data))])
BATCH_SIZE = 32
epochs = 150

In [ ]:
model, loss, acc, test_acc = construct.train_test(200, 32, train, test, torch.nn.CrossEntropyLoss(), 'Adam', model)

In [ ]:
plt.plot(test_acc,figsize=(12,8))

In [ ]:
plt.plot(acc)

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(acc, linewidth=3, color='red')
plt.grid()
plt.xlabel('epochs', fontsize=(24), fontweight='bold')
plt.ylabel('train accuracy', fontsize=(24), fontweight='bold')
plt.xticks(fontsize=(16))
plt.yticks(fontsize=(16))

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(loss, linewidth=3)
plt.grid()
plt.xlabel('epochs', fontsize=(24), fontweight='bold')
plt.ylabel('loss', fontsize=(24), fontweight='bold')
plt.xticks(fontsize=(16))
plt.yticks(fontsize=(16))
plt.text(color='white')

In [3]:
class myModel(torch.nn.Module):
    def __init__(self):
        """
        Initializes CNN. Here we just define layer shapes that we call in the forward func
        """
        super().__init__()

        self.conv1 = torch.nn.Conv2d(in_channels = 3, 
                               out_channels = 6, 
                               kernel_size = 5,
                                    bias = False)
                
        #Convultion layer 2. See above
        self.conv2 = torch.nn.Conv2d(in_channels = 6, 
                               out_channels = 12, 
                               kernel_size = 5,
                                    bias = False)
        
        self.fc_1 = torch.nn.Linear(39 * 39 * 12, 256)
        self.fc_2 = torch.nn.Linear(256, 2)
        self.drop = torch.nn.Dropout(p=.2)
        self.batch1 = torch.nn.BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch2 = torch.nn.BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            
    def forward(self, x):
        """
        Function that performs all the neural network forward calculation i.e.
        takes image data from the input of the neural network to the output
        """
        
        x = self.conv1(x)
        x = self.batch1(x)
        x = torch.nn.functional.max_pool2d(x, kernel_size = 2)
        x = torch.nn.functional.leaky_relu(x)
        x = self.conv2(x)
        x = self.batch2(x)
        x = torch.nn.functional.max_pool2d(x, kernel_size = 4)
        x = torch.nn.functional.leaky_relu(x)
        x = x.view(x.shape[0], -1)  
        x = self.fc_1(x) 
        x = torch.nn.functional.leaky_relu(x)
        x = self.drop(x)
        x = self.fc_2(x) 
        return x

model = myModel()

In [4]:
k = 5
epochs = 5
BATCH_SIZE = 32
criterion = torch.nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transforms = torchvision.transforms.Compose([
                            torchvision.transforms.ToPILImage(),
                            torchvision.transforms.RandomRotation((-180,180)),
                            torchvision.transforms.CenterCrop((325)),
                            torchvision.transforms.ToTensor()
                                      ])
optimizer = torch.optim.Adam(model.parameters(), lr=.001)

x = construct.k_fold(k, epochs, BATCH_SIZE, transforms, criterion, model, optimizer, device, image_dir)
(models, losses, train_accs, naive_accs, test_accs) = x

NameError: name 'KFold' is not defined

In [ ]:
kf = KFold(n_splits=5, shuffle = True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
models = []
losses = []
train_accs = []
test_accs = []
naive_accs = []
epochs = 150
BATCH_SIZE = 32
criterion = torch.nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transforms = torchvision.transforms.Compose([
                            torchvision.transforms.ToPILImage(),
                            torchvision.transforms.RandomRotation((-180,180)),
                            torchvision.transforms.CenterCrop((325)),
                            torchvision.transforms.ToTensor()
                                      ])



for train_idx, test_idx in kf.split(DATA):
    curr_model = myModel()
    train = DATA.iloc[train_idx].reset_index()
    test = DATA.iloc[test_idx].reset_index()
    train_data = prep.tenX_dataset(train, 'data/raman_images', transform=transforms)
    test_data = prep.tenX_dataset(test, 'data/raman_images', transform=transforms)
    cnn, train_loss, train_acc = construct.train(epochs, BATCH_SIZE, 
                                                 train_data, criterion, 'Adam', curr_model, device, with_eval)
    models.append(cnn)
    train_accs.append(train_acc)
    losses.append(train_loss)
    images, labels, predictions, weights, test_acc = construct.get_predictions(BATCH_SIZE, cnn, test_data)
    test_accs.append(test_acc)
    naive_accs.append((labels[:,1] == 0).float().sum()/len(predictions))
    

In [ ]:
n_splits =3
kf = KFold(n_splits=n_splits, shuffle=True)